In [49]:
import os
import zipfile
import collections
import shutil

def find_broken_encoding_folder(parent_dir):
    """
    parent_dir 내에서 인코딩이 깨진 것으로 보이는 폴더를 찾습니다.
    """
    for item_name in os.listdir(parent_dir):
        item_path = os.path.join(parent_dir, item_name)
        if os.path.isdir(item_path):
            try:
                item_name.encode('utf-8')
            except UnicodeEncodeError:
                print(f"  - 인코딩이 깨진 폴더 감지: {item_name}")
                return item_path
    return None

def move_contents_up(source_folder, target_folder):
    """
    source_folder 안의 모든 파일과 폴더를 target_folder로 이동시킵니다.
    """
    print(f"  - '{os.path.basename(source_folder)}'의 내용물을 상위 폴더로 이동시키는 중...")
    for item_name in os.listdir(source_folder):
        source_item = os.path.join(source_folder, item_name)
        destination_item = os.path.join(target_folder, item_name)
        if os.path.exists(destination_item):
            print(f"    - 경고: '{destination_item}'이 이미 존재하여 건너뜁니다.")
            continue
        shutil.move(source_item, target_folder)
    try:
        os.rmdir(source_folder)
        print(f"  - '{os.path.basename(source_folder)}' 폴더를 삭제했습니다.")
    except OSError as e:
        print(f"  - 폴더 삭제 실패: {e}")

def unzip_and_organize(initial_zip_path, output_base_dir, duplicate_to_path=None):
    """
    zip 파일 압축 해제, 정리, 성공 시 원본 zip 삭제 (kfood.zip 제외), 그리고 최종 폴더 복제.
    """
    initial_extract_dir = os.path.join(output_base_dir, os.path.splitext(os.path.basename(initial_zip_path))[0])
    queue = collections.deque([(os.path.abspath(initial_zip_path), initial_extract_dir)])
    successful_unzip = True

    while queue:
        zip_path, extract_dir = queue.popleft()

        if not os.path.exists(zip_path) or not zipfile.is_zipfile(zip_path):
            print(f"오류: '{zip_path}'는 유효한 zip 파일이 아닙니다.")
            successful_unzip = False
            continue

        try:
            os.makedirs(extract_dir, exist_ok=True)
            print(f"압축 해제 중: {zip_path} -> '{extract_dir}'")

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                for member in zip_ref.infolist():
                    is_utf8 = member.flag_bits & 0x800
                    if is_utf8: decoded_name = member.filename
                    else:
                        try: decoded_name = member.filename.encode('cp437').decode('utf-8')
                        except UnicodeDecodeError: decoded_name = member.filename
                    
                    target_path = os.path.join(extract_dir, decoded_name)
                    if member.is_dir(): os.makedirs(target_path, exist_ok=True)
                    else:
                        parent_dir = os.path.dirname(target_path)
                        os.makedirs(parent_dir, exist_ok=True)
                        with zip_ref.open(member) as s, open(target_path, "wb") as t: t.write(s.read())

            print("  - 성공적으로 압축 해제되었습니다.")
            
            broken_folder_path = find_broken_encoding_folder(extract_dir)
            if broken_folder_path:
                move_contents_up(broken_folder_path, extract_dir)
            
            for dirpath, _, filenames in os.walk(extract_dir):
                for filename in filenames:
                    if filename.lower().endswith(".zip"):
                        new_zip_path = os.path.join(dirpath, filename)
                        new_extract_dir = os.path.splitext(new_zip_path)[0]
                        print(f"  - 중첩된 zip 파일 발견, 큐에 추가: {new_zip_path}")
                        queue.append((new_zip_path, new_extract_dir))

        except Exception as e:
            print(f"오류 발생 ({zip_path}): {e}")
            successful_unzip = False

    if successful_unzip:
        # =================== 핵심 수정 부분 ===================
        # --- 원본 zip 파일 삭제 로직 ---
        # 현재 처리한 zip 파일의 이름이 'kfood.zip'이 아닌 경우에만 삭제
        if os.path.basename(initial_zip_path).lower() != 'kfood.zip':
            try:
                os.remove(initial_zip_path)
                print(f"\n작업 완료 후 원본 zip 파일 '{initial_zip_path}'을(를) 삭제했습니다.")
            except OSError as e:
                print(f"\n원본 zip 파일 '{initial_zip_path}' 삭제 실패: {e}")
        else:
            print(f"\n'{initial_zip_path}' 파일은 삭제하지 않고 보존합니다.")
        # ======================================================

        # --- 최종 폴더 복제 로직 ---
        if duplicate_to_path:
            final_duplicate_path = os.path.join(duplicate_to_path, os.path.basename(initial_extract_dir))
            print(f"\n최종 폴더 복제 시작: '{initial_extract_dir}' -> '{final_duplicate_path}'")
            try:
                if os.path.exists(final_duplicate_path):
                    shutil.rmtree(final_duplicate_path)
                    print(f"  - 기존 폴더 '{final_duplicate_path}'을(를) 삭제했습니다.")
                
                shutil.copytree(initial_extract_dir, final_duplicate_path)
                print("  - ✅ 폴더 복제가 성공적으로 완료되었습니다.")
            except Exception as e:
                print(f"  - ❌ 폴더 복제 중 오류 발생: {e}")
        # ======================================================

# --- 코드 실행 부분 ---

zip_file_to_unzip = '/Users/ihanjo/Documents/kfood.zip'
destination_folder = '한국음식 예측/데이터/data/kfood_original' # 임시 압축 해제 및 정리 공간
duplicate_destination_folder = '한국음식 예측/data/kfood' # 최종 결과물을 복제할 위치

unzip_and_organize(zip_file_to_unzip, destination_folder, duplicate_to_path=duplicate_destination_folder)
print("\n모든 작업이 완료되었습니다.")

오류: '/Users/ihanjo/Documents/kfood.zip'는 유효한 zip 파일이 아닙니다.

모든 작업이 완료되었습니다.
